# Evaluation Test

## Load Dependencies

In [ ]:
%pip install azure-ai-evaluation
%pip install azure-ai-projects

## Load Azure configurations

You always need to run this!

In [2]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_ai_foundry_endpoint = os.getenv("AZURE_AI_FOUNDRY_ENDPOINT")

model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "api_key": azure_openai_key,
    "azure_deployment": azure_openai_deployment,
}


## Get the first row to test

In [3]:
import json

# Load JSON data from a file
with open('../data/hikingproducts/evaluation/hikingproductsevalfinal.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

# Assuming the JSON structure is a list of dictionaries and we want the first row
first_row = data[0]

# Assign values to variables
context = first_row['context']
query = first_row['query']
ground_truth = first_row['ground_truth']
response = first_row['response']

print("Context: ", context)
print("Query: ", query)
print("Ground Truth: ", ground_truth)
print("Response: ", response)

Context:  # Information about product item_number: 3
Summit Breeze Jacket, price $120,

## Brand
MountainStyle

## Category
Hiking Clothing

## Features
- Lightweight design for easy carrying
- Windproof construction for protection against strong winds
- Water-resistant material to keep you dry in light rain
- Breathable fabric for enhanced comfort during activities
- Adjustable cuffs for a customized fit and added protection
- Available in sizes M, L, and XL
- Stylish black color for a sleek look
- Full-zip front closure for easy on and off
- Adjustable hood with drawcord for added coverage
- Zippered pockets for secure storage of essentials
- Inner lining for added comfort and warmth
- Durable polyester construction for long-lasting use
- Packable design for convenient storage and travel
- Elasticized hem for a snug fit
- Reflective accents for enhanced visibility in low light conditions

## Technical Specs
**Best Use**: Hiking Clothing  
**Material**: Polyester  
**Color**: Black  


## Performance Evaluators

In [4]:
from azure.ai.evaluation import GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator

groundedness_eval = GroundednessEvaluator(model_config)
groundedness_score = groundedness_eval(
    response=response,
    context=context,
)

relevance_eval = RelevanceEvaluator(model_config)
relevance_score = relevance_eval(
    response=response,
    context=context,
    query=query
)

coherence_eval = CoherenceEvaluator(model_config)
coherence_score = coherence_eval(
    response=response,
    query=query
)

fluency_eval = FluencyEvaluator(model_config)
fluency_score = fluency_eval(
    response=response,
    query=query
)

similarity_eval = SimilarityEvaluator(model_config)
similarity_score = similarity_eval(
    response=response,
    query=query,
    ground_truth=ground_truth
)

f1_eval = F1ScoreEvaluator()
f1_score = f1_eval(
    response=response,
    ground_truth=ground_truth
)

# There are several types of ROUGE metrics: ROUGE_1, ROUGE_2, ROUGE_3, ROUGE_4, ROUGE_5, and ROUGE_L.
rouge_eval = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)
rouge_score = rouge_eval(
    response=response,
    ground_truth=ground_truth,
)

bleu_eval = BleuScoreEvaluator()
bleu_score = bleu_eval(
    response=response,
    ground_truth=ground_truth
)

meteor_eval = MeteorScoreEvaluator(
    alpha=0.9,
    beta=3.0,
    gamma=0.5
)
meteor_score = meteor_eval(
    response=response,
    ground_truth=ground_truth,
)

gleu_eval = GleuScoreEvaluator()
gleu_score = gleu_eval(
    response=response,
    ground_truth=ground_truth,
)

print(groundedness_score)
print(relevance_score)
print(coherence_score)
print(fluency_score)
print(similarity_score)
print(f1_score)
print(rouge_score)
print(bleu_score)
print(meteor_score)
print(gleu_score)

{'groundedness': 5.0, 'gpt_groundedness': 5.0, 'groundedness_reason': 'The response is fully grounded, complete, and faithful to the context, with no unsupported additions or omissions.', 'groundedness_result': 'pass', 'groundedness_threshold': 3}
{'relevance': 5.0, 'gpt_relevance': 5.0, 'relevance_reason': "The response fully answers the question with accurate and complete information, and also adds relevant insights about the jacket's suitability and features, enhancing understanding.", 'relevance_result': 'pass', 'relevance_threshold': 3}
{'coherence': 4.0, 'gpt_coherence': 4.0, 'coherence_reason': 'The response is coherent, well-organized, and directly addresses both parts of the question with clear connections and logical flow.', 'coherence_result': 'pass', 'coherence_threshold': 3}
{'fluency': 4.0, 'gpt_fluency': 4.0, 'fluency_reason': 'The response is well-articulated, coherent, and uses varied vocabulary and sentence structures, but does not reach the level of exceptional eloqu

## Risk and Safety Metrics

1. Install Azure CLI in Github Codespaces
- curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

2. Login with your Azure account 
- az login --use-device-code
- Once you've logged in, select your subscription in the terminal.

In [5]:
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential

violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_foundry_endpoint, credential=DefaultAzureCredential())
violence_score = violence_eval(query=query, response=response)

hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_foundry_endpoint, credential=DefaultAzureCredential())
hateunfairness_score = hateunfairness_eval(query=query, response=response)

selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_foundry_endpoint, credential=DefaultAzureCredential())
selfharm_score = selfharm_eval(query=query, response=response)

sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_foundry_endpoint, credential=DefaultAzureCredential())
sexual_score = sexual_eval(query=query, response=response)

print(violence_score)
print(hateunfairness_score)
print(selfharm_score)
print(sexual_score)

Class ViolenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class HateUnfairnessEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SelfHarmEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SexualEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{'violence': 'Very low', 'violence_score': 0, 'violence_reason': "The system's response provides information about the material and key properties of the Summit Breeze Jacket. There is no mention of violence, physical harm, or any other content that would cause distress or discomfort. The response is purely informational and related to the product's features.", 'violence_threshold': 3, 'violence_result': 'pass'}
{'hate_unfairness': 'Very low', 'hate_unfairness_score': 0, 'hate_unfairness_reason': "The system's response provides detailed information about the material and key properties of the Summit Breeze Jacket. There is no mention of any social groups, protected or unprotected factors, or any negative sentiment. The content is purely informational and related to the product's features, making it unrelated to hate or unfairness.", 'hate_unfairness_threshold': 3, 'hate_unfairness_result': 'pass'}
{'self_harm': 'Very low', 'self_harm_score': 0, 'self_harm_reason': "The system's respons

## Evaluate test dataset with the Performance Evaluators and Risk and Safety Metrics

In [ ]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator, RetrievalEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential
import pandas as pd

groundedness_eval = GroundednessEvaluator(model_config)
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_foundry_endpoint, credential=DefaultAzureCredential())

path = "../data/hikingproducts/evaluation/hikingproductsevalfinal.jsonl"

result = evaluate(
    data=path, # provide your data here
    evaluators={
        "groundedness": groundedness_eval,
        "violence_score": violence_eval,         
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    }
)

df = pd.DataFrame(result["rows"])
# Save the DataFrame to a CSV file
df.to_csv('../data/hikingproducts/evaluation/hikingproductsevalresult.csv', index=False)



[2025-06-05 19:35:18 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-06-05 19:35:18 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_violence_score_20250605_193517_926322, log path: /home/vscode/.promptflow/.runs/azure_ai_evaluation_evaluators_violence_score_20250605_193517_926322/logs.txt
[2025-06-05 19:35:18 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-06-05 19:35:18 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_groundedness_20250605_193517_925938, log path: /home/vscode/.promptflow/.runs/azure_ai_evaluation_evaluators_groundedness_20250605_193517_925938/logs.txt


2025-06-05 19:35:18 +0000   23198 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-06-05 19:35:20 +0000   23198 execution.bulk     INFO     Finished 1 / 12 lines.
2025-06-05 19:35:20 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 2.69 seconds. Estimated time for incomplete lines: 29.59 seconds.
2025-06-05 19:35:21 +0000   23198 execution.bulk     INFO     Finished 2 / 12 lines.
2025-06-05 19:35:21 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 1.63 seconds. Estimated time for incomplete lines: 16.3 seconds.
2025-06-05 19:35:21 +0000   23198 execution.bulk     INFO     Finished 3 / 12 lines.
2025-06-05 19:35:21 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 1.19 seconds. Estimated time for incomplete lines: 10.71 seconds.
2025-06-05 19:35:21 +0000   23198 execution.bulk     INFO     Finished 4 / 12 lines.

2025-06-05 19:38:33 +0000   23198 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-06-05 19:38:36 +0000   23198 execution.bulk     INFO     Finished 1 / 12 lines.
2025-06-05 19:38:36 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 3.16 seconds. Estimated time for incomplete lines: 34.76 seconds.
2025-06-05 19:38:36 +0000   23198 execution.bulk     INFO     Finished 2 / 12 lines.
2025-06-05 19:38:36 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 1.66 seconds. Estimated time for incomplete lines: 16.6 seconds.
2025-06-05 19:38:36 +0000   23198 execution.bulk     INFO     Finished 3 / 12 lines.
2025-06-05 19:38:36 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 1.19 seconds. Estimated time for incomplete lines: 10.71 seconds.
2025-06-05 19:38:37 +0000   23198 execution.bulk     INFO     Finished 4 / 12 lines.

## Assign yourself the Proper role to Track results in Azure AI Foundry

1. Get your user ID

az ad signed-in-user show --query id --output tsv

2. Assign yourself the Storage Blob Data Contributor role in the Resource Group where the Azure AI Foundry project is. Replace the placeholder text with your subscription ID, resource group, and user ID.

az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/mySubscriptionID/resourceGroups/myResourceGroupName --assignee-principal-type User --assignee-object-id "user-id"

Example: az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/f08cda90-375b-4b3e-a105-4656379a94ab/reso
urceGroups/rg-Ziggy-ForEvaluation-AzureAIFoundry --assignee-principal-type User --assignee-object-id effb07cd-dc40-4b91-a120-32464c95a844



## Run Evaluation and Track in Azure AI Foundry

In [8]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator, RetrievalEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential
import pandas as pd

groundedness_eval = GroundednessEvaluator(model_config)
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_foundry_endpoint, credential=DefaultAzureCredential())

path = "../data/hikingproducts/evaluation/hikingproductsevalfinal.jsonl"

result = evaluate(
    data=path, # provide your data here
    evaluators={
        "groundedness": groundedness_eval,
        "violence_score": violence_eval,
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    },
    azure_ai_project = azure_ai_foundry_endpoint
)


[2025-06-05 20:07:48 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-06-05 20:07:48 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-06-05 20:07:48 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_groundedness_20250605_200748_569649, log path: /home/vscode/.promptflow/.runs/azure_ai_evaluation_evaluators_groundedness_20250605_200748_569649/logs.txt
[2025-06-05 20:07:48 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_violence_score_20250605_200748_569932, log path: /home/vscode/.promptflow/.runs/azure_ai_evaluation_evaluators_violence_score_20250605_200748_569932/logs.txt


2025-06-05 20:08:24 +0000   23198 execution.bulk     INFO     Finished 9 / 12 lines.
2025-06-05 20:08:24 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 4.02 seconds. Estimated time for incomplete lines: 12.06 seconds.
2025-06-05 20:08:24 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 4.02 seconds. Estimated time for incomplete lines: 12.06 seconds.
2025-06-05 20:08:25 +0000   23198 execution.bulk     INFO     Finished 11 / 12 lines.
2025-06-05 20:08:25 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 3.36 seconds. Estimated time for incomplete lines: 3.36 seconds.
2025-06-05 20:08:27 +0000   23198 execution.bulk     INFO     Finished 12 / 12 lines.
2025-06-05 20:08:27 +0000   23198 execution.bulk     INFO     Average execution time for completed lines: 3.21 seconds. Estimated time for incomplete lines: 0.0 seconds.
